In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession 
from pyspark import SparkConf,SparkContext
import os

conf = SparkConf()
conf.setMaster('local')
conf.setAppName('pyspark-shell')
conf.set('spark.driver.host', '127.0.0.1')
os.environ['PYSPARK_PYTHON'] = '/Users/ccompain/.pyenv/versions/miniconda3-latest/bin/python' # Needs to be explicitly provided as env. Otherwise workers run Python 2.7
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'

sc = SparkContext(conf=conf)
sc.setLogLevel("WARN") 

In [2]:
r=sc.parallelize([("A","B"),("B","C"),("B","D"),("D","E"),("F","G"),("G","H")],2)
r.collect()

[('A', 'B'), ('B', 'C'), ('B', 'D'), ('D', 'E'), ('F', 'G'), ('G', 'H')]

In [3]:
def CCF_Iterate_reduce(pair):
  key, values = pair
  global accum
  min = key
  valueL = []
  for value in values:
    if value < min:
       min = value
    valueL.append(value)
  if min < key:
    yield((key, min))
    for value in valueL:
      if min != value:
        accum.add(1)
        yield((value, min))

In [4]:
log4jLogger = sc._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)


In [5]:
new_pair_flag = True
iteration = 0
accum = sc.accumulator(0)
dedupJob = r

In [6]:
while new_pair_flag:
    iteration += 1
    newPair = False
    accum.value = 0

    # CCF-iterate (MAP)
    mapJob = dedupJob.flatMap(lambda e: (e,e[::-1]))
    #print(mapJob.collect())

    # CCF-iterate (REDUCE)
    reduceJob = mapJob.groupByKey().flatMap(lambda pair: CCF_Iterate_reduce(pair)).sortByKey()

    # CCF-dedup 
    dedupJob = reduceJob.distinct()

    # Force the RDD evalusation
    tmp = dedupJob.count()

    # Prepare next iteration
    #mapJob = dedupJob
    new_pair_flag = bool(accum.value)

    #LOGGER.warn("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag))
    print("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag))


Iteration :1 ---  newPair : True
Iteration :2 ---  newPair : True
Iteration :3 ---  newPair : True
Iteration :4 ---  newPair : True
Iteration :5 ---  newPair : False


In [7]:
results = list(map(lambda e: e[::-1], mapJob.collect()))
print(results)
for k in results:
    print("Component id:", k[0], "| Number of nodes: ", len(k[1]) +1)

[('A', 'B'), ('B', 'A'), ('A', 'D'), ('D', 'A'), ('A', 'E'), ('E', 'A'), ('F', 'G'), ('G', 'F'), ('F', 'H'), ('H', 'F'), ('A', 'C'), ('C', 'A')]
Component id: A | Number of nodes:  2
Component id: B | Number of nodes:  2
Component id: A | Number of nodes:  2
Component id: D | Number of nodes:  2
Component id: A | Number of nodes:  2
Component id: E | Number of nodes:  2
Component id: F | Number of nodes:  2
Component id: G | Number of nodes:  2
Component id: F | Number of nodes:  2
Component id: H | Number of nodes:  2
Component id: A | Number of nodes:  2
Component id: C | Number of nodes:  2
